In [98]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import math
import numpy as np
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch.optim as optim


#### Transformer Encoder

In [174]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    

class Transformer(nn.Module):
    def __init__(self, input_dim, d_model, heads, num_layers,dropout, max_len, output_classes):
        super().__init__()
        self.encoder_layer= nn.TransformerEncoderLayer(d_model=d_model, nhead=heads)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.positional_encoder = PositionalEncoding(d_model=d_model, dropout=dropout, max_len=max_len)
        self.linear = nn.Linear(in_features=d_model, out_features=output_classes)
        self.fc = nn.Linear(input_dim, d_model)
        self.softmax = nn.Softmax(dim = 1)
        
    def forward(self, x):
        x1 = self.fc(x)
        x1 = self.positional_encoder(x1)
        x2 = self.encoder(x1)
        out = self.linear(x2)
        # out = self.softmax(out)
        return out
        


kinematic_features = 22 #input_dim
dmodel = 64 #dmodel
heads = 4
num_layers = 2
dropout = 0.1
max_len = 100
output_classes = 7

# src = torch.rand(10, 10, kinematic_features)

model = Transformer(kinematic_features,dmodel, heads,num_layers,dropout,max_len,output_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)


IndentationError: expected an indented block (3795282794.py, line 44)

#### Dataset Class

In [123]:
gesture_labels_directory = './new_dataset/peg_transfer/peg_transfer/labeled/gestures/'
window_size = 10 # 10 frame windows (kinematic data)
enite_dataset = []
counter = 0



class KinematicDataset(Dataset):
    
    def __init__(self,data_directory, window_size):
        
        self.directory = data_directory
        self.window_size = window_size
        self.le = preprocessing.LabelEncoder()
        self.enc = preprocessing.OneHotEncoder(sparse_output=False)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(self.device)
        (self.X, self.Y) = self._load_data()
        self.X = self.X.to(self.device)
        self.Y = self.Y.to(self.device)
        
        
    def _load_data(self):
        x = []
        y = []
        self.windows_per_file = []
        
        for filename in os.listdir(self.directory):
            f = os.path.join(self.directory, filename)
            # checking if it is a file
            if os.path.isfile(f):
                kinematics_data = pd.read_csv(f)
                kin_data = kinematics_data.iloc[:,:-1]
                kin_label = kinematics_data.iloc[:,-1]
                
                x.append(kin_data.values)
                y.append(kin_label.values)
                
                
                
                if len(self.windows_per_file) == 0:
                    self.windows_per_file = [len(kin_label) - self.window_size + 1]
                else:
                    self.windows_per_file.append(self.windows_per_file[-1] + len(kin_label) - self.window_size + 1)
        
        self.le.fit(y[0])
        y = [self.le.transform(yi) for yi in y]
        
        y = [yi.reshape(len(yi), 1) for yi in y]
        
        y = [self.enc.fit_transform(yi) for yi in y]
        
        x = np.concatenate(x)
        y = np.concatenate(y)
        
        x = torch.from_numpy(x)
        y = torch.from_numpy(y)
        
        x = x.to(torch.float32)
        return (x, y)
    
    def __len__(self):
        # this should return the size of the dataset
        return self.Y.shape[0]-window_size
    
    def __getitem__(self, idx):
        # this should return one sample from the dataset
        features = self.X[idx : idx + window_size]
        target = self.Y[idx : idx + window_size]
        return features, target
                    
                    


In [25]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

print(integer_encoded)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

['cold' 'cold' 'warm' 'cold' 'hot' 'hot' 'warm' 'cold' 'warm' 'hot']
[0 0 2 0 1 1 2 0 2 1]
[[0]
 [0]
 [2]
 [0]
 [1]
 [1]
 [2]
 [0]
 [2]
 [1]]
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
['cold']


/home/kesharaw/anaconda3/envs/rsurgery/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [124]:
# train_data, test_data = 

batch_size = 32

dataset = KinematicDataset(gesture_labels_directory,10)

# use 20% of training data for validation
train_set_size = int(len(dataset) * 0.8)
valid_set_size = len(dataset) - train_set_size
print(train_set_size, valid_set_size)

seed = torch.Generator().manual_seed(42)
train_set, test_set = torch.utils.data.random_split(dataset, [train_set_size, valid_set_size],generator=seed)


trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=False, num_workers=0)

testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=False, num_workers=0)

print(train_set.dataset.X.shape, test_set.dataset.X.shape)
print(train_set.dataset.Y.shape, test_set.dataset.Y.shape)


cuda
119279 29820
torch.Size([149109, 22]) torch.Size([149109, 22])
torch.Size([149109, 7]) torch.Size([149109, 7])


In [126]:
# loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [131]:
epochs = 10

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        
        # print(inputs[0,:,:].shape)
        # print(outputs[0,:,:].shape)
        # print(labels[0,:,:].shape)
        
        # print(inputs.shape)
        # print(outputs.shape)
        # print(labels.shape)
        
        outputs = (torch.flatten(outputs, start_dim=0, end_dim=1))
        labels = (torch.flatten(labels, start_dim=0, end_dim=1))
        
        # break
        # outputs = torch.transpose(outputs,1,2)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        # for name, param in model.named_parameters():
        #     print(name, param.grad)

        # if(i > 2):
        #     break
        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,    50] loss: 0.023
[1,   100] loss: 0.016
[1,   150] loss: 0.013
[1,   200] loss: 0.011
[1,   250] loss: 0.010
[1,   300] loss: 0.010
[1,   350] loss: 0.009
[1,   400] loss: 0.009
[1,   450] loss: 0.008
[1,   500] loss: 0.009
[1,   550] loss: 0.008
[1,   600] loss: 0.008
[1,   650] loss: 0.008
[1,   700] loss: 0.007
[1,   750] loss: 0.008
[1,   800] loss: 0.008
[1,   850] loss: 0.007
[1,   900] loss: 0.007
[1,   950] loss: 0.007
[1,  1000] loss: 0.007
[1,  1050] loss: 0.007
[1,  1100] loss: 0.007
[1,  1150] loss: 0.007
[1,  1200] loss: 0.007
[1,  1250] loss: 0.007
[1,  1300] loss: 0.007
[1,  1350] loss: 0.007
[1,  1400] loss: 0.007
[1,  1450] loss: 0.007
[1,  1500] loss: 0.006
[1,  1550] loss: 0.007
[1,  1600] loss: 0.007
[1,  1650] loss: 0.007
[1,  1700] loss: 0.006
[1,  1750] loss: 0.006
[1,  1800] loss: 0.006
[1,  1850] loss: 0.006
[1,  1900] loss: 0.007
[1,  1950] loss: 0.006
[1,  2000] loss: 0.006
[1,  2050] loss: 0.007
[1,  2100] loss: 0.006
[1,  2150] loss: 0.006
[1,  2200] 

In [128]:
MODEL_NAME = "MODEL_V0.pth"
MODEL_SAVE_PATH = "./models/"+MODEL_NAME

torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH)

In [172]:
#Test

# y_pred = model(list(iter(test_set)))

for i,data in enumerate(testloader):
    x,y = data
    
    y_pred = model(x)
    
    
    y_pred = (torch.flatten(y_pred, start_dim=0, end_dim=1))
    gt = (torch.flatten(y, start_dim=0, end_dim=1))
    
    
    y_pred = y_pred.cpu()
    y_pred = y_pred.detach().numpy()
    
        
    gt = gt.cpu()
    gt = gt.detach().numpy()
    
    gt_f = np.argmax(gt,axis=1)
    y_pred_f = np.argmax(y_pred,axis=1)
    
    accuracy = np.mean(gt_f == y_pred_f) * 100
    print("Accuracy: ", accuracy)

Accuracy:  90.9375
Accuracy:  84.6875
Accuracy:  91.25
Accuracy:  95.9375
Accuracy:  93.125
Accuracy:  95.9375
Accuracy:  93.125
Accuracy:  88.75
Accuracy:  89.6875
Accuracy:  92.1875
Accuracy:  92.8125
Accuracy:  88.4375
Accuracy:  92.8125
Accuracy:  87.8125
Accuracy:  92.5
Accuracy:  90.3125
Accuracy:  95.0
Accuracy:  91.875
Accuracy:  89.375
Accuracy:  94.6875
Accuracy:  94.6875
Accuracy:  93.4375
Accuracy:  88.125
Accuracy:  95.3125
Accuracy:  98.4375
Accuracy:  91.875
Accuracy:  90.9375
Accuracy:  92.5
Accuracy:  89.6875
Accuracy:  95.0
Accuracy:  90.3125
Accuracy:  90.625
Accuracy:  93.125
Accuracy:  93.125
Accuracy:  88.4375
Accuracy:  93.4375
Accuracy:  92.1875
Accuracy:  95.625
Accuracy:  88.4375
Accuracy:  93.125
Accuracy:  93.75
Accuracy:  95.0
Accuracy:  93.75
Accuracy:  98.75
Accuracy:  89.6875
Accuracy:  85.9375
Accuracy:  95.0
Accuracy:  95.0
Accuracy:  91.875
Accuracy:  93.4375
Accuracy:  92.8125
Accuracy:  93.4375
Accuracy:  91.25
Accuracy:  91.25
Accuracy:  94.6875
Ac